# Benchmark of hydrogen wavefunction integration

In [1]:
import timeit

import numpy as np

from numerov.rydberg import RydbergState

test_cases: list[tuple[str, int, int, bool]] = [
    # species, n, l, use_njit
    ("H", 1, 0, True),
    ("H", 1, 0, False),
]

In [2]:
def run_benchmark(number: int = 10) -> list[dict]:
    """Run benchmark for different quantum states.

    Args:
        number: Number of times to run each test for averaging

    """
    # run the integration once to compile the numba function
    species, n, l, use_njit = test_cases[0]
    atom = RydbergState(species, n, l, j=l + 0.5, _use_njit=True)
    atom.integrate()

    results = []
    for species, n, l, use_njit in test_cases:
        # Setup the test function
        stmt = (
            "atom = RydbergState(species, n, l, j=l+0.5, _use_njit=use_njit)\n"
            + "atom.create_grid(steps=10_000)\n"
            + "atom.integrate()"
        )

        # Time the integration multiple times and take average/std
        globals_dict = {"RydbergState": RydbergState, "species": species, "n": n, "l": l, "use_njit": use_njit}
        times = timeit.repeat(stmt=stmt, number=1, repeat=number, globals=globals_dict)
        avg_time = np.mean(times)
        std_time = np.std(times)

        results.append({"species": species, "n": n, "l": l, "use_njit": use_njit, "time": avg_time, "std": std_time})

    return results

In [3]:
results = run_benchmark(number=2)

print("\nBenchmark Results:")
print("-" * 70)
print(f"{'species':>8} {'n':>3} {'l':>3} {'use_njit':>10} {'time (ms)':>10} {'std (ms)':>10}")
print("-" * 70)
for r in results:
    print(
        f"{r['species']:>8} {r['n']:>3} {r['l']:>3} {r['use_njit']!s:>10} "
        f"{r['time'] * 1000:10.2f} {r['std'] * 1000:10.2f}"
    )

xmin=0.010000000000000002 was not chosen good (sum_small_z=0.004976178614469415), and the wavefunction is positive at the inner boundary, so we could not fix it.
xmin=0.010000000000000002 was not chosen good (sum_small_z=0.005510004753634187), and the wavefunction is positive at the inner boundary, so we could not fix it.
xmin=0.010000000000000002 was not chosen good (sum_small_z=0.005510004753634187), and the wavefunction is positive at the inner boundary, so we could not fix it.
xmin=0.010000000000000002 was not chosen good (sum_small_z=0.005510004753634187), and the wavefunction is positive at the inner boundary, so we could not fix it.
xmin=0.010000000000000002 was not chosen good (sum_small_z=0.005510004753634187), and the wavefunction is positive at the inner boundary, so we could not fix it.



Benchmark Results:
----------------------------------------------------------------------
 species   n   l   use_njit  time (ms)   std (ms)
----------------------------------------------------------------------
       H   1   0       True       1.99       0.12
       H   1   0      False      23.83       0.20
